In [ ]:
%matplotlib widget
# imports & matplotlib options
import pandas as pd
import numpy as np
import ipywidgets as widgets
import cv2
import collections
import random
from shutil import copy
from pathlib import Path
from datetime import datetime
from collections import OrderedDict
from ipywidgets import interact, interactive, IntProgress, Label, VBox
from IPython.display import display
from IPython.display import Image as ipyImage
from matplotlib import pyplot as plt
import matplotlib as mpl
import img_qc.img_qc as img_qc
from skimage.measure import compare_ssim as ssim
from PIL import Image


from io import BytesIO

print(f'\n{25 * "*_*"}\nLast run on {datetime.now().strftime("%Y-%m-%d %I:%M:%S %p")}')

In [ ]:
image_path = Path('/Users/jmoor167/Documents/GitHub/utk_ProjectCeres/data/agrutesc/0012_004268_000006_0001.tif')
image = Image.open(image_path)

In [ ]:
plt.imshow(image, cmap='gray')

In [ ]:
data_dir_path = Path('data/')
page_1_paths_list = sorted(data_dir_path.glob('agrutesc/*.tif'))

# delete macOS '.' index files
regenerate_paths_list = False
for path in page_1_paths_list:
    if path.name.startswith('.'):
        path.unlink()
        regenerate_paths_list = True

if regenerate_paths_list:
    page_1_paths_list = sorted(data_dir_path.glob('agrutesc/*.tif'))

print(f'{len(page_1_paths_list)} images in page 1 paths list')


print(f'\n{25 * "*_*"}\nLast run on {datetime.now().strftime("%Y-%m-%d %I:%M:%S %p")}')

In [ ]:
page_1_paths_list

In [ ]:
def array2image(array):
    """Display a 2- or 3-d numpy arrayay as an image."""
    if array.ndim == 2:
        format, cmap = 'png', mpl.cm.gray
    elif array.ndim == 3:
        format, cmap = 'jpg', None
    else:
        raise ValueError("Only 2- or 3-d arrayays can be displayed as images.")
    # Don't let matplotlib autoscale the color range so we can control overall luminosity
    vmax = 255 if array.dtype == 'uint8' else 1.0
    with BytesIO() as buffer:
        mpl.image.imsave(buffer, array, format=format, cmap=cmap, vmin=0, vmax=vmax)
        out = buffer.getvalue()
    clear_output = True
    return ipyImage(out)

        
def quick_imshow(bgr_image):
    if len(bgr_image.shape) > 2:
        bgr_image = cv2.cvtColor(bgr_image, cv2.COLOR_BGR2RGB)
        cmap = None
    else:
        cmap = 'gray'
    plt.imshow(bgr_image, cmap=cmap), plt.show()
    
    return

In [ ]:
page_1_paths_dictionary = {path.name: path for path in page_1_paths_list}
image_name = page_1_paths_list[22].name
path = page_1_paths_dictionary[image_name]
image_key = path.name
image = cv2.imread(str(path))
height, width = image.shape[:2]

def choose_image(image_name):
    # Let's store the result in the global `image` that we can then use in our image editor below
    global image
    try:
        image_path = page_1_paths_dictionary[image_name.new]
    except AttributeError:  # 'str' object has no attribute 'new' if it's the first image loaded
        image_path = page_1_paths_dictionary[image_name]
    print(image_path.name)
    image = cv2.imread(str(image_path))
    _temp_image_path = data_dir_path.joinpath('_temp_image_canDelete.jpg')
    cv2.imwrite(str(_temp_image_path), image)
    image = cv2.imread(str(_temp_image_path))
    # if _temp_image_path.is_file():
    #     display(ipyImage(_temp_image_path))
    return image

def update_height(height_range):
    global y1, y2
    try:
        y1, y2 = height_range.new
    except AttributeError:  # first run through error
        y1, y2 = height_range
    crop_box = x1, y1, x2, y2
    return crop_box

def update_width(width_range):
    global x1, x2
    try:
        x1, x2 = width_range.new
    except AttributeError:  # first run through error
        x1, x2 = width_range
    crop_box = x1, y1, x2, y2
    return crop_box

def draw_crop_box(crop_box):
    x1, y1, x2, y2 = crop_box
    drawing = cv2.rectangle(image.copy(), (x1, y1), (x2, y2), (0,0,255), 3)
    # plt.imshow(drawing)
    return drawing

def show_image(image_name, height_range, width_range):
    global image, output_widget
    image = choose_image(image_name)
    crop_box = update_height(height_range)
    crop_box = update_width(width_range)
    drawing = draw_crop_box(crop_box)
    with output_widget:
        output_widget.clear_output(wait=True)
        display(my_widget)
        display(ipyImage(drawing))
    
    

# Create widgets
x1 = 100
x2 = width - 100
y1 = 100
y2 = height - 100
title_text_widget = widgets.Text(value=image_key)
image_name_widget = widgets.Dropdown(
    options=list(page_1_paths_dictionary), value=image_key)
width_widget = widgets.IntRangeSlider(value=(x1, x2), min=0, max=width, continuous_update=False)
height_widget = widgets.IntRangeSlider(value=(y1, y2), min=0, max=height, continuous_update=False)

image_name_widget.observe(choose_image, 'value')
width_widget.observe(update_width, 'value')
height_widget.observe(update_height, 'value')



# interact(choose_image, image_name=[x.name for x in page_1_paths_list])
# interact(update_height, height_range=height_widget)


my_widget = interactive(show_image, image_name=image_name_widget, height_range=height_widget, width_range=width_widget)

output_widget = widgets.Output(layout={'border': '1px solid black', 'height': '1000px'})

In [ ]:
with output_widget:
    display(my_widget)
output_widget

In [ ]:
from ipywidgets import Output, IntSlider, VBox
from IPython.display import display
out = Output()
slider = IntSlider()
slider2 = IntSlider()
textbox = widgets.Text(value='This should update with a number')

@interact # new decorator - not sure if this is a good name for it...
def update_textbox(change):
    textbox.value = str(change.new)
    
@out.capture() # new decorator - not sure if this is a good name for it...
def update_slider(change):
    if change.new.isdigit():
        slider.value = int(change.new)

@out.capture()
def update_slider2(change):
    if change.new


update_slider.observe(change, 'value')
update_slider2.observe(change, 'value')
update_textbox.observe(change, 'value')

display(VBox([slider, textbox, slider2]))

In [ ]:
# this is my attempt to try and make a crop-value widget!

page_1_paths_dictionary = {path.name: path for path in page_1_paths_list}
image_name = page_1_paths_list[22].name
path = page_1_paths_dictionary[image_name]
image_key = path.name
image = cv2.imread(str(path))
height, width = image.shape[:2]

# Create widgets
images = widgets.Dropdown(
    options=list(page_1_paths_dictionary), value=image_key)
x1 = 100
x2 = width - 100
y1 = 100
y2 = height - 100
title_text_widget = widgets.Text(value=image_key)
crop_x_widget = widgets.IntRangeSlider(value=(x1, x2), min=0, max=width)
crop_y_widget = widgets.IntRangeSlider(value=(y1, y2), min=0, max=height)


app = widgets.AppLayout(header=widgets.Button(), top_right=widgets.Button(),
                        bottom_left=widgets.Button(), bottom_right=widgets.Button())
display(app)


#load a new image based ont eh selected image from the page_1_paths_dictionary value
# TODO: update crop box optins based on the new image

def update_crop(*args):
    image_path = page_1_paths_dictionary[images.value]
    image = cv2.imread(str(image_path))
    height, width = image.shape[:2]

    
def show_image(image_path):
    image_path = page_1_paths_dictionary[images.value]
    image = cv2.imread(str(image_path))
    
    plt.imshow(image), plt.show()
    
    # can't embed a tif so convert to PIL image on the fly
    
    # _temp_image_path = data_directory_path.joinpath('_temp_image_canDelete.jpg')
    # cv2.imwrite(str(_temp_image_path), image)
    # if _temp_image_path.is_file():
    #     display(ipyImage(_temp_image_path))
        
        
# tie the image dropdown to the show_image value
images.observe(show_image, 'value')


        
_ = interact(show_image, image_path=images)